In [19]:
import requests
import pandas as pd
from datetime import datetime, timezone

In [ ]:
#https://api.worldbank.org/v2/country/all/indicator/SL.UEM.TOTL.NE.ZS?date=2024:2024
#https://api.worldbank.org/v2/countries/all/indicators/SL.UEM.TOTL.ZS 
# use this to check api/metadata: https://api.worldbank.org/v2/countries/all/indicator/SL.UEM.TOTL.ZS?date=2023:2024&format=json&page=6
indicator="SL.UEM.TOTL.ZS"
date_range = "2022:2023"
#base_url = f"https://api.worldbank.org/v2/countries/all/indicators/{indicator}?{date_range}&format=json"
base_url = f"https://api.worldbank.org/v2/countries/all/indicators/{indicator}?"
params = {
    "date": date_range,
    "format": "json",
    "page": 1  # Start at page 1
}

#response_data = []
all_data = []

while True:
    response = requests.get(base_url, params=params)
    response_data = response.json()
    
    if len(response_data) < 2 or not response_data[1]:  # Check if there's data
        break
    
    all_data.extend(response_data[1])  # Add current page data to all_data
    
    # Update parameters for the next page
    params["page"] += 1

df_unemployment = pd.json_normalize(data=all_data)

print(df_unemployment)

In [19]:
# Separate the metadata and records
#metadata = all_data[0]
#records = all_data[1]
#df_unemployment = pd.json_normalize(data=all_data)


In [ ]:
df_unemployment.columns

df_unemployment.dtypes
df_unemployment.info()

In [ ]:
df_unemployment['country.value'].astype(str)

In [ ]:
df_unemployment.head() 



In [ ]:
#print(df_unemployment["countryiso3code"])
df_unemployment.loc[df_unemployment['countryiso3code'] == 'SGP'] 


In [17]:

df_unemployment.to_excel("data/suph_export_unemp_from api.xlsx")